<a href="https://colab.research.google.com/github/fabiocorreaf/Assistente_Virtual/blob/main/Assistente_Virtual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install gtts
!pip install wikipedia
!pip install pyjokes
!pip install winshell
!pip install pygame

In [2]:
from gtts import gTTS
from IPython.display import Audio, display
from io import BytesIO

def speak(texto=''):
  lingua = "en"
  arquivo = gTTS(text=texto, lang=lingua, slow=True)
  mp3_fp = BytesIO()
  arquivo.write_to_fp(mp3_fp)
  mp3_fp.seek(0)
  display(Audio(mp3_fp.read(), rate=22050, autoplay=True))

In [3]:
from gtts import gTTS
from IPython.display import Audio, display
import whisper
import base64
import os
import datetime
from google.colab import output
from IPython.display import display, HTML
import time
transcricao = None

def record_audio():
    """Exibe os botões de gravação e processa o áudio gravado no Colab."""
    display(HTML('''
        <script>
            var audioStream;
            var audioRecorder;
            var audioData = [];

            async function startRecording() {
                const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
                audioStream = stream;
                audioRecorder = new MediaRecorder(stream);

                audioRecorder.ondataavailable = function(event) {
                    audioData.push(event.data);
                };

                audioRecorder.onstop = function() {
                    const blob = new Blob(audioData, { type: 'audio/wav' });

                    // Converte para Data URL e envia para o Python
                    const reader = new FileReader();
                    reader.readAsDataURL(blob);
                    reader.onloadend = function() {
                        const dataUrl = reader.result;
                        google.colab.kernel.invokeFunction('process_audio', [dataUrl], {});
                    };
                };

                audioRecorder.start();
                document.getElementById("startButton").disabled = true;
                document.getElementById("stopButton").disabled = false;
            }

            function stopRecording() {
                audioRecorder.stop();
                audioStream.getTracks().forEach(track => track.stop());
                document.getElementById("stopButton").disabled = true;
            }
        </script>

        <button id="startButton" onclick="startRecording()">🎙️ Iniciar Gravação</button>
        <button id="stopButton" onclick="stopRecording()" disabled>⏹️ Parar Gravação</button>
    '''))

def process_audio(data_url):
    """Salva o áudio no Colab, transcreve e exibe o texto."""
    header, encoded = data_url.split(",", 1)
    data = base64.b64decode(encoded)

    # Nome do arquivo baseado no timestamp
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    filename = f"audio_{timestamp}.wav"
    filepath = os.path.join("/content/", filename)

    with open(filepath, "wb") as f:
        f.write(data)

    print(f"✅ Áudio salvo como: {filename}")

    # Carrega o modelo Whisper para transcrição
    model = whisper.load_model("base")
    result = model.transcribe(filepath)

    text = result["text"]

    # Exibe a transcrição imediatamente
    print("\n🎤 **Transcrição:", text )
    return text

# Registra a função de processamento para ser chamada pelo JavaScript
output.register_callback('process_audio', process_audio)

# Exibe os botões para iniciar a gravação
record_audio()



✅ Áudio salvo como: audio_20250222002613.wav


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)



🎤 **Transcrição:  Bom dia!


In [ ]:

def respond(transcricao):
  while transcricao == None:
    time.sleep = (1)
    speak("Text from get audio " + transcricao)
    if 'youtube' in transcricao:
        speak("What do you want to search for?")
        keyword = record_audio()
        if keyword!= '':
            url = f"https://www.youtube.com/results?search_query={keyword}"
            webbrowser.get().open(url)
            speak(f"Here is what I have found for {keyword} on youtube")
    elif 'search' in transcricao:
        speak("What do you want to search for?")
        query = record_audio()
        if query !='':
            result = wikipedia.summary(query, sentences=3)
            speak("According to wikipedia")
            print(result)
            speak(result)
    elif 'joke' in transcricao:
        speak(pyjokes.get_joke())
    elif 'empty recycle bin' in transcricao:
        winshell.recycle_bin().empty(confirm=False, show_progress=False, sound=True)
        speak("Recycle bin emptied")
    elif 'what time' in transcricao:
        strTime = datetime.today().strftime("%H:%M %p")
        print(strTime)
        speak(strTime)
    elif 'play music' in transcricao or 'play song' in transcricao:
        speak("Now playing...")
        music_dir = "C:\\Users\\UserName\\Downloads\\Music\\" #add your music directory here..
        songs = os.listdir(music_dir)
        #counter = 0
        print(songs)
        playmusic(music_dir + "\\" + songs[0])
    elif 'stop music' in transcricao:
        speak("Stopping playback.")
        stopmusic()
    elif 'exit' in transcricao:
        speak("Goodbye, till next time")
        exit()
    elif TypeError:
      pass
#play music
def playmusic(song):
    mixer.init()
    mixer.music.load(song)
    mixer.music.play()
#stop music
def stopmusic():
    mixer.music.stop()
    #let's try it
#text = get_audio()
#speak(text)
while True:
    speak("I am listening...")
    transcricao = record_audio()
    respond(transcricao)